## Model generator main file

In [ ]:
import numpy as np
import re
import unidecode
import os
import pickle
import sys
import csv
import random

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection     import GridSearchCV
from sklearn.metrics             import average_precision_score
from sklearn.model_selection     import KFold, cross_val_score

import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints           import maxnorm
from keras                       import regularizers
from keras                       import Input, Model, Sequential
from keras.layers                import Flatten, TimeDistributed, CuDNNGRU, CuDNNLSTM, Bidirectional, Activation, TimeDistributed, Dense, RepeatVector, Embedding, Dropout, BatchNormalization
from keras.layers.recurrent      import LSTM, GRU, SimpleRNN
from keras.utils                 import np_utils
from keras.callbacks             import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.optimizers            import Adam, RMSprop, SGD

Import data and make folders

In [ ]:
data_path = 'data'
temp_path = data_path + '/temp'
backup_path = data_path + '/backup'
weights_path = data_path + '/weights'
train_history = data_path + '/train_history'
resource_path = 'Pickled_data'
test_set_path = resource_path + '/test_sets'

if not os.path.isdir(data_path):
    os.mkdir(data_path)
if not os.path.isdir(temp_path):
    os.mkdir(temp_path)
if not os.path.isdir(backup_path):
    os.mkdir(backup_path)
if not os.path.isdir(weights_path):
    os.mkdir(weights_path)
if not os.path.isdir(train_history):
    os.mkdir(train_history)
if not os.path.isdir(test_set_path):
    os.mkdir(train_history)

#joint_angle_data.pickle
with open(resource_path + '/100_percent_path4_high_auginput.pickle', 'rb') as file:
    joint_angle_data = pickle.load(file)
    file.close()

#power_data.pickle
with open(resource_path + '/100_percent_path4_high_poweroutput.pickle', 'rb') as file:
    power_data = pickle.load(file)
    file.close()
    
#joint_angle_data = joint_angle_data[:,:-6]
print('===== Check sizes =====')
print('joint_angle_data shape is: ')
print(joint_angle_data.shape)
print('power_data shape is: ')
print(power_data.shape)
print('First value needs to be equal!')

## Recurrent neural networks

Different model functions

In [ ]:
def mean_square_error(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))

In [ ]:
def setModelToBasicLSTM(self, multiple=False):
    n_timesteps, n_features, n_outputs = self.X_train.shape[1], self.X_train.shape[2], self.Y_train.shape[1]
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    rmsprop = RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
    sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    # define model
    self.model = Sequential()
    self.model.add(Bidirectional(CuDNNLSTM(128*2, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', input_shape = (n_timesteps, n_features))))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(128*2, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(128*2, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(128*2, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(128*2, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(128*2, return_sequences = multiple, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    if multiple:
        self.model.add(TimeDistributed(Dense(128, activation='relu')))
        self.model.add(TimeDistributed(Dense(n_outputs)))
    else:
        self.model.add(Dense(128, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero'))
        self.model.add(Dense(128, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero'))
        self.model.add(Dense(n_outputs))
    self.model.compile(loss='mean_absolute_percentage_error', optimizer=rmsprop, metrics=[mean_square_error])   

In [ ]:
def setModelToGridSearch(self, multiple=False, layer_type=CuDNNLSTM, hidden_units_RNN=128, hidden_units_dense=64, num_mid_layers = 3, optimizer='adam', init_mode='random_uniform', activation='relu', dropout_rate=0.2, num_dense_layers=1):
    n_timesteps, n_features, n_outputs = self.X_train.shape[1], self.X_train.shape[2], self.Y_train.shape[-1]
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    rmsprop = RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
    sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    # define model
    self.model = Sequential()
    self.model.add(Bidirectional(layer_type(hidden_units_RNN, return_sequences = True, kernel_initializer = init_mode, bias_initializer = 'zero', input_shape = (n_timesteps, n_features))))
    for n in range(num_mid_layers):
        self.model.add(Bidirectional(layer_type(hidden_units_RNN, return_sequences = True, kernel_initializer = init_mode, bias_initializer = 'zero')))
        self.model.add(Dropout(dropout_rate))
        
    self.model.add(Bidirectional(layer_type(hidden_units_RNN, return_sequences = multiple, kernel_initializer = init_mode, bias_initializer = 'zero')))
    self.model.add(Dropout(dropout_rate))
    
    if multiple:
        for n in range(num_dense_layers):
            self.model.add(TimeDistributed(Dense(hidden_units_dense, activation=activation, kernel_initializer = 'random_uniform', bias_initializer = 'zero'))) #, kernel_constraint=maxnorm(weight_constraint)
        self.model.add(TimeDistributed(Dense(n_outputs)))
    else:
        for n in range(num_dense_layers):
            self.model.add(Dense(hidden_units_dense, activation=activation, kernel_initializer = 'random_uniform', bias_initializer = 'zero')) #, kernel_constraint=maxnorm(weight_constraint)
        self.model.add(Dense(n_outputs))
        
    self.model.compile(loss='mean_squared_error', optimizer=rmsprop, metrics=["accuracy"])   
    return self.model

In [ ]:
def setModelToFFNN(self):
    n_timesteps, n_features, n_outputs = self.X_train.shape[1], self.X_train.shape[2], self.Y_train.shape[-1]
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    rmsprop = RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
    sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    # define model
    inputs = Input(shape=(n_timesteps,n_features))
    x = Dense(256, activation='relu')(inputs)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    prediction = Dense(n_outputs, activation='relu')(x)
    
    self.model = Model(inputs, prediction)
    self.model.compile(loss='mean_absolute_percentage_error', optimizer=rmsprop, metrics=[mean_square_error])   

In [ ]:
def setModelToFinalModel(self, multiple=False):
    n_timesteps, n_features, n_outputs = self.X_train.shape[1], self.X_train.shape[2], self.Y_train.shape[-1]
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    rmsprop = RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
    sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    # define model
    self.model = Sequential()
    self.model.add(Bidirectional(CuDNNLSTM(256, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', input_shape = (n_timesteps, n_features))))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(256, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(256, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(256, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(256, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Bidirectional(CuDNNLSTM(256, return_sequences = multiple, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    if multiple:
        self.model.add(TimeDistributed(Dense(64, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
        self.model.add(TimeDistributed(Dense(64, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
        self.model.add(TimeDistributed(Dense(n_outputs)))
    else:
        self.model.add(Dense(64, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero'))
        self.model.add(Dense(64, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero'))
        self.model.add(Dense(n_outputs))
    self.model.compile(loss='mean_absolute_percentage_error', optimizer=rmsprop, metrics=['mean_absolute_percentage_error',mean_square_error])   

Other functions

In [ ]:
def series_to_supervised(dataX, dataY, n_input, n_output, multiple=False):
    #Output: [batchs, n_timesteps, n_features]
    df_X = pd.DataFrame(dataX)
    df_Y = pd.DataFrame(dataY)
    X = np.ndarray((df_X.shape[0]-n_input, n_input, df_X.shape[1]) )
    if multiple:
        Y = np.ndarray((df_Y.shape[0]-n_input, n_input, n_output))
    else:
        Y = np.ndarray((df_Y.shape[0]-n_input, n_output))

    for n in range(0,df_X.shape[0]-n_input-1,1):
        for m in range(n_input-1,-1, -1):
            X[n,m,:] = df_X.loc[n+m:n+m,:]
            
    if multiple:
        for n in range(0,df_Y.shape[0]-n_input-1,1):
            for m in range(n_input-1,-1, -1):
                Y[n,m,:] = df_Y.loc[n+m:n+m,:]
    else:
        for n in range(0,df_Y.shape[0]-n_input-1,1):
            for m in range(n_output-1,-1, -1):
                Y[n,:] = df_Y.loc[n+m:n+m,:]
            
    return X, Y

In [ ]:
def setUpData(self, seq_length, dataX, dataY, n_test_ratio, multiple=False):
    self.X_train = []
    self.Y_train = []
    self.X_test = []
    self.Y_test = []
    self.seq_length = seq_length
    
    X, Y = series_to_supervised(dataX, dataY, multiple=multiple, n_input=seq_length, n_output=dataY.shape[1])
    
    self.X_train = X[:int(len(X)*(1-n_test_ratio)), :, :]
    self.Y_train = Y[:int(len(Y)*(1-n_test_ratio)), :]
    self.X_test = X[int(len(X)*(1-n_test_ratio)):, :, :]
    self.Y_test = Y[int(len(Y)*(1-n_test_ratio)):, :]
    

In [ ]:
def setUpMultiData(self, seq_length, dataX_paths, dataY_paths, match_list_vector, num_loaded, limited_data=False, multiple=False):
    
    self.seq_length = seq_length
    num_loaded_loop = 0
    for n in range(len(dataX_paths)):
        print('Work: ' + str(n+1) + ' of ' + str(len(dataX_paths)))
        #joint_angle_data.pickle
        with open(dataX_paths[n], 'rb') as file:
            joint_angle_data = pickle.load(file)
            file.close()

        #power_data.pickle
        with open(dataY_paths[match_list_vector[n]], 'rb') as file:
            power_data = pickle.load(file)
            file.close()

        if(int(joint_angle_data.shape[0]) != int(power_data.shape[0])):
            print('Files not equal in sizes! could not use.')
            continue

        X, Y = series_to_supervised(joint_angle_data, power_data, multiple=multiple, 
                                    n_input=seq_length, n_output=power_data.shape[-1])
        
        num_loaded_loop = num_loaded_loop + 1
        if( num_loaded_loop == 1):
            self.X_train = X
            self.Y_train = Y
            print(self.X_train.shape)
            print(self.Y_train.shape)
            continue
    
        randomNum = random.sample(range(0, len(X)), 6000)
        if multiple:
            #for num in range(5000):
            self.X_train = np.concatenate((self.X_train,X[randomNum,:,:]))
            self.Y_train = np.concatenate((self.Y_train,Y[randomNum,:,:]))
            print(self.X_train.shape)
            print(self.Y_train.shape)
        else:
            #for num in range(5000):
            self.X_train = np.concatenate((self.X_train,X[randomNum,:,:]))
            self.Y_train = np.concatenate((self.Y_train,Y[randomNum,:]))
            print(self.X_train.shape)
            print(self.Y_train.shape)

        if(num_loaded_loop>num_loaded):
            print('"num_loaded" reached stopping')
            break

In [ ]:
def trainModel(self, name):
    # File path for model
    filepath = weights_path + "/weights-" + name + "-{epoch:02d}-{loss:.4f}.hdf5"
    # Callbacks functions
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32)
    mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
    
    # Train the model
    epochs = 500           #Maximum number of epochs to run
    batch_size = 128    #Size of training data batch
    Val_split = 0.1       #Procentage of training data to use as validation data
    history = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, validation_split=Val_split, callbacks=[es, tb, mc])
    
    # Save the model
    filename = "model_" + name + ".hdf5"
    self.model.save_weights(weights_path + '/' + filename)
    # Save the history
    filename = "history_" + name + ".pickle"
    with open(train_history + '/' + filename, 'wb') as f:
        pickle.dump(history.history, f)
        f.close()

In [ ]:
def loadPreProcessedData(self, Data_X_name, Data_Y_name):
    #Data_X_name
    with open(Data_X_name, 'rb') as file:
        self.X_train = pickle.load(file)
        file.close()

    #Data_Y_name
    with open(Data_Y_name, 'rb') as file:
        self.Y_train = pickle.load(file)
        file.close()

In [ ]:
def preTrain(self):
    epochs = 1           #Maximum number of epochs to run
    batch_size = 128     #Size of training data batch
    Val_split = 0.1       #Procentage of training data to use as validation data
    history = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, validation_split=Val_split)
    

Model generator

In [ ]:
class Generator:

    set_up_data = setUpData
    set_up_multi_data = setUpMultiData
    train_model = trainModel
    load_pre_processed_data = loadPreProcessedData
    pre_train = preTrain
    
    set_model_to_Basic_LSTM = setModelToBasicLSTM
    set_model_to_Grid_Search = setModelToGridSearch
    set_model_to_FFNN = setModelToFFNN
    set_model_to_final_model = setModelToFinalModel
    

## Training the network

Test training FFNN

In [ ]:
gen = Generator()
seqLleng = 25 #12ms/leng unit
n_test_ratio = 0.02
print((power_data.shape))
gen.set_up_data(seqLleng, joint_angle_data, power_data, n_test_ratio, multiple=False)
print(type(gen.X_train))
print(gen.X_train.shape)
print(gen.Y_train.shape)
print(gen.X_test.shape)
print(gen.Y_test.shape)
gen.set_model_to_FFNN()
name = 'test_run_FFNN'
gen.train_model(name = name)

Test training old data Basic LSTM

In [ ]:
old_data = False
if old_data:
    #joint_angle_data.pickle
    with open('data/augmented_input.pickle', 'rb') as file:
        joint_angle_data = pickle.load(file)
        file.close()

    #power_data.pickle
    with open('data/power_data.pickle', 'rb') as file:
        power_data = pickle.load(file)
        file.close()
else:
    #joint_angle_data.pickle
    with open(resource_path + '/100_percent_path4_high_auginput.pickle', 'rb') as file:
        joint_angle_data = pickle.load(file)
        file.close()

    #power_data.pickle
    with open(resource_path + '/100_percent_path4_high_poweroutput.pickle', 'rb') as file:
        power_data = pickle.load(file)
        file.close()
    #joint_angle_data = joint_angle_data[:,:-6]

gen = Generator()
seqLleng = 100 #12ms/leng unit
n_test_ratio = 0.02
multiple = False
print((power_data.shape))
print((joint_angle_data.shape))
gen.set_up_data(seqLleng, joint_angle_data, power_data, n_test_ratio, multiple=False)
print(gen.X_train.shape)
print(gen.Y_train.shape)
print(gen.X_test.shape)
print(gen.Y_test.shape)
gen.set_model_to_Basic_LSTM(multiple=multiple)
name = 'test_run_Basic_LSTM'
gen.train_model(name = name)

Test training current loaded data - basic LSTM

In [ ]:
gen = Generator()
seqLleng = 100 #12ms/leng unit
n_test_ratio = 0.00
multiple = False
print((joint_angle_data.shape))
print((power_data.shape))
gen.set_up_data(seqLleng, joint_angle_data, power_data, n_test_ratio, multiple=False)
print(gen.X_train.shape)
print(gen.Y_train.shape)
print(gen.X_test.shape)
print(gen.Y_test.shape)
gen.set_model_to_Basic_LSTM(multiple=multiple)
name = 'test_run_Basic_LSTM'
gen.train_model(name = name)

Test training all files together (can be modified to load just some certain types) basic LSTM

In [ ]:
gen = Generator()
seqLleng = 100 #12ms/leng unit
n_test_ratio = 0.02
multiple = False
#print((power_data.shape))

folder_path = 'Pickled_data/'
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(folder_path):
    path = os.path.join(folder_path, name)
    if "_high_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_high_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_high_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1
        
print(joint_angle_data_path_list[1])
print(power_data_path_list[match_list_vector[1]])
num_loaded = 3
n_test = 0
gen.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, n_test, num_loaded, multiple=multiple)
print(gen.X_train.shape)
print(gen.Y_train.shape)
print(gen.X_test.shape)
print(gen.Y_test.shape)
gen.set_model_to_Basic_LSTM(multiple=multiple)
name = 'test_run_Basic_LSTM'
gen.train_model(name = name)

Hyperparamter Grid Search

In [ ]:
gen_gs = Generator()
# Variables
seqLength = 100 #12ms/leng unit
n_test_ratio = 0
multiple = False
# Set up the data
print('Setting up data')
gen_gs.set_up_data(seqLength, joint_angle_data, power_data, n_test_ratio, multiple=multiple)
# create model
print('Creating keras Classifier')
epochs = 20
batch_size = 32
model = KerasClassifier(build_fn=gen_gs.set_model_to_Grid_Search, epochs=epochs, batch_size=batch_size, verbose=1)
# define the grid search parameters
multiple = [multiple]
layer_type = [CuDNNLSTM]; #[CuDNNLSTM, CuDNNGRU]
hidden_units_RNN = [128, 256]
hidden_units_dense = [64, 128]
num_dense_layers = [1, 2]
num_mid_layers = [0, 2, 4]
#optimizer = ['adam']
#learn_rate = [0.01]
#momentum = [0]
#init_mode = ['random_uniform']
#activation = ['relu']
dropout_rate = [0.2]
#weight_constraint = [0]
# Make dictionary
param_grid = dict(layer_type =layer_type,
                  hidden_units_dense=hidden_units_dense,
                  hidden_units_RNN=hidden_units_RNN,
                  num_mid_layers=num_mid_layers,
                  dropout_rate=dropout_rate,
                  num_dense_layers=num_dense_layers)
# Grid Search
print('Grid Search Starting')
k_fold = KFold(n_splits=2)
grid = GridSearchCV(estimator=model, cv=k_fold, scoring='mean_squared_error', param_grid=param_grid, n_jobs=1, verbose=100)
print('X: ' + str(gen_gs.X_train.shape))
print('Y: ' + str(gen_gs.Y_train.shape))
grid_result = grid.fit(gen_gs.X_train, gen_gs.Y_train)

## Prediction Results

Results from Grid Search, which summarizes the data and saves the data

In [ ]:
# Summarize the results from the Grid Search
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Save the results from the Grid Search
mylist = [['mean_test_score', 'std_test_score', 'params']]
for mean, stdev, param in zip(means, stds, params):
    mylist.append([mean, stdev, param])
    
with open('Grid_Search_Result.csv', 'w') as myfile:
    #wr = csv.writer(myfile, dialect='excel')
    for row in mylist:
        for column in row:
            myfile.write('%s;' % column)
        myfile.write('\n')
    myfile.close()
print('Done Saving')

Prediction using train and test data and plot the result (OLD DATA)

In [ ]:
predictions_train = gen.model.predict(gen.X_train)
predictions_test = gen.model.predict(gen.X_test)


plt.figure(figsize=(12,6))
fig1 = plt.subplot()
x1 = range(len(predictions_train))
fig1.plot(x1[100:1000], gen.Y_train[100:1000],  label='true',  marker='o',linewidth=0.3, markersize=2)
fig1.plot(x1[100:1000], predictions_train[100:1000],  label='pred',  marker='o',linewidth=0.3, markersize=2)
fig1.legend();
fig1.set_ylabel('Power[W]')
fig1.set_xlabel('Time')
fig1.set_title('Train data prediction');

plt.figure(figsize=(12,6))
fig2 = plt.subplot()
x2 = range(len(predictions_test))
fig2.plot(x2, gen.Y_test,  label='true',  marker='o',linewidth=0.3, markersize=2)
fig2.plot(x2, predictions_test,  label='pred',  marker='o',linewidth=0.3, markersize=2)
fig2.legend();
fig2.set_ylabel('Power[W]')
fig2.set_xlabel('Time')
fig2.set_title('Test data prediction');

fig1.savefig('foo.png')

Prediction using train and test data and plot the result (NEW DATA)

In [ ]:
predictions_train = gen.model.predict(gen.X_train)
predictions_test = gen.model.predict(gen.X_test)

plt.figure(figsize=(12,6))
fig1 = plt.subplot()
x1 = range(len(predictions_train))
fig1.plot(x1[100:1000], gen.Y_train[100:1000],  label='true',  marker='o',linewidth=0.3, markersize=2)
fig1.plot(x1[100:1000], predictions_train[100:1000],  label='pred',  marker='o',linewidth=0.3, markersize=2)
fig1.legend();
fig1.set_ylabel('Power[W]')
fig1.set_xlabel('Time')
fig1.set_title('Train data prediction');

plt.figure(figsize=(12,6))
fig2 = plt.subplot()
x2 = range(len(predictions_test))
fig2.plot(x2, gen.Y_test,  label='true',  marker='o',linewidth=0.3, markersize=2)
fig2.plot(x2, predictions_test,  label='pred',  marker='o',linewidth=0.3, markersize=2)
fig2.legend();
fig2.set_ylabel('Power[W]')
fig2.set_xlabel('Time')
fig2.set_title('Test data prediction');

## Final training 

In [ ]:
# test out different sequence lengths
seq_lengths =  [10,25,50,75,100,125,150,175,200]
multiple = False
n_test_ratio = 0
#joint_angle_data.pickle
with open(resource_path + '/100_percent_path4_high_auginput.pickle', 'rb') as file:
    joint_angle_data = pickle.load(file)
    file.close()

#power_data.pickle
with open(resource_path + '/100_percent_path4_high_poweroutput.pickle', 'rb') as file:
    power_data = pickle.load(file)
    file.close()
    
for n in range(len(seq_lengths)):
    print('Testing seqLength: ' + str(seq_lengths[n]))
    gen_final = Generator()
    print('Setting up data')
    gen_final.set_up_data(seq_lengths[n], joint_angle_data, power_data, n_test_ratio, multiple=False)
    gen_final.set_model_to_final_model(multiple=multiple)
    print(gen_final.X_train.shape)
    print(gen_final.Y_train.shape)
    name = 'SeqLength_test_Final_Model_' + str(seq_lengths[n])
    print('Begining to train')
    gen_final.train_model(name = name)

For singel output

In [ ]:
# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(2*len(joint_angle_data_path_list)))
gen = Generator()
seqLleng = 50 
n_test = 0
num_loaded = len(joint_angle_data_path_list)+1
multiple = False
print('Begining to load')
gen.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, num_loaded, multiple=multiple)
print(gen.X_train.shape)
print(gen.Y_train.shape)

print('Saving the data')
#joint_angle_data.pickle
with open('data/temp/joint_angle_data_training_data.pickle', 'wb') as file:
    pickle.dump(gen.X_train, file)
    file.close()

#power_data.pickle
with open('data/temp/power_data_training_data.pickle', 'wb') as file:
    pickle.dump(gen.Y_train, file)
    file.close()

In [ ]:
# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seqLleng = 100 
n_test = 0
num_loaded = len(joint_angle_data_path_list)+1
multiple = False
print('Begining to load')
gen_final.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, num_loaded, multiple=multiple)
print(gen_final.X_train.shape)
print(gen_final.Y_train.shape)

## Setup for final training
#gen_final = Generator()
#seqLleng = 50 
#multiple = False
## Load pre pre processed data
#print('Loading Pre Processed Data')
#gen_final.load_pre_processed_data('data/temp/joint_angle_data_training_data.pickle','data/temp/power_data_training_data.pickle')
#print(gen_final.X_train.shape)
#print(gen_final.Y_train.shape)
print('Initializing model')
gen_final.set_model_to_final_model(multiple=multiple)

# The training
n_iterations = 3
for n in range(n_iterations):
    print('Iteration: ' + str(n+1))
    name = 'Final_Model_Iteration_' + str(n+1)
    gen_final.train_model(name = name)


In [ ]:
#For hard drives with less RAM

# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seq_length = 100 
multiple = False

# The training
n_iterations = 3
n_test_ratio = 0
first_run = True
for n in range(n_iterations):
    num_loaded_loop = 0
    print('Iteration: ' + str(n+1))
    for m in range(len(joint_angle_data_path_list)):
        print('Begining to load')
        print('Work: ' + str(m+1) + ' of ' + str(len(joint_angle_data_path_list)))
        #joint_angle_data.pickle
        with open(joint_angle_data_path_list[m], 'rb') as file:
            joint_angle_data = pickle.load(file)
            file.close()

        #power_data.pickle
        with open(power_data_path_list[match_list_vector[m]], 'rb') as file:
            power_data = pickle.load(file)
            file.close()

        if(int(joint_angle_data.shape[0]) != int(power_data.shape[0])):
            print('Files not equal in sizes! could not use.')
            continue
        
        gen_final.set_up_data(seq_length, joint_angle_data, power_data, n_test_ratio, multiple=multiple)
        print(gen_final.X_train.shape)
        print(gen_final.Y_train.shape)
        if (first_run):
            first_run = False
            print('Initializing model')
            gen_final.set_model_to_final_model(multiple=multiple)
        
        name = 'Final_Model_singleOut_Iteration_' + str(n+1) + '_trainingNum_' + str(m+1)
        gen_final.train_model(name = name)


For multiple output

In [ ]:
#For hard drives with less RAM

# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seq_length = 100 
multiple = True

# The training
n_iterations = 2
n_test_ratio = 0
first_run = True
for n in range(n_iterations):
    num_loaded_loop = 0
    print('Iteration: ' + str(n+1))
    for m in range(len(joint_angle_data_path_list)):
        print('Begining to load')
        print('Work: ' + str(m+1) + ' of ' + str(len(joint_angle_data_path_list)))
        #joint_angle_data.pickle
        with open(joint_angle_data_path_list[m], 'rb') as file:
            joint_angle_data = pickle.load(file)
            file.close()

        #power_data.pickle
        with open(power_data_path_list[match_list_vector[m]], 'rb') as file:
            power_data = pickle.load(file)
            file.close()

        if(int(joint_angle_data.shape[0]) != int(power_data.shape[0])):
            print('Files not equal in sizes! could not use.')
            continue
        
        gen_final.set_up_data(seq_length, joint_angle_data, power_data, n_test_ratio, multiple=multiple)
        print(gen_final.X_train.shape)
        print(gen_final.Y_train.shape)
        if (first_run):
            first_run = False
            print('Initializing model')
            gen_final.set_model_to_final_model(multiple=multiple)
        
        name = 'Final_Model_multipeOut_Iteration_' + str(n+1) + '_trainingNum_' + str(m+1)
        gen_final.train_model(name = name)


For test, do not use

In [ ]:
#For hard drives with less RAM

# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seq_length = 100 
multiple = False

# The training
n_iterations = 2
n_test_ratio = 0
first_run = True
for n in range(n_iterations):
    num_loaded_loop = 0
    print('Iteration: ' + str(n+1))
    for m in range(len(joint_angle_data_path_list)):
        print('Begining to load')
        print('Work: ' + str(m+1) + ' of ' + str(len(joint_angle_data_path_list)))
        #joint_angle_data.pickle
        with open(joint_angle_data_path_list[m], 'rb') as file:
            joint_angle_data = pickle.load(file)
            file.close()

        #power_data.pickle
        with open(power_data_path_list[match_list_vector[m]], 'rb') as file:
            power_data = pickle.load(file)
            file.close()

        if(int(joint_angle_data.shape[0]) != int(power_data.shape[0])):
            print('Files not equal in sizes! could not use.')
            continue
        
        gen_final.set_up_data(seq_length, joint_angle_data, power_data, n_test_ratio, multiple=multiple)
        print(gen_final.X_train.shape)
        print(gen_final.Y_train.shape)
        if (first_run):
            first_run = False
            print('Initializing model')
            gen_final.set_model_to_FFNN(multiple=multiple)
        
        name = 'Final_Model_FNN_Iteration_' + str(n+1) + '_trainingNum_' + str(m+1)
        gen_final.train_model(name = name)


In [ ]:
# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seqLleng = 100 
n_test = 0
num_loaded = len(joint_angle_data_path_list)+1
multiple = True
print('Begining to load')
gen_final.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, num_loaded, multiple=multiple)
print(gen_final.X_train.shape)
print(gen_final.Y_train.shape)

## Setup for final training
#gen_final = Generator()
#seqLleng = 50 
#multiple = False
## Load pre pre processed data
#print('Loading Pre Processed Data')
#gen_final.load_pre_processed_data('data/temp/joint_angle_data_training_data.pickle','data/temp/power_data_training_data.pickle')
#print(gen_final.X_train.shape)
#print(gen_final.Y_train.shape)
print('Initializing model')
gen_final.set_model_to_final_model(multiple=multiple)

# The training
n_iterations = 3
for n in range(n_iterations):
    print('Iteration: ' + str(n+1))
    name = 'Final_Model_Iteration_multiple_' + str(n+1)
    gen_final.train_model(name = name)


## DESSA ÄR DE TRE MODELERNA SOM SKA KÖRAS

In [ ]:
# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seqLleng = 100 
n_test = 0
num_loaded = len(joint_angle_data_path_list)+1
multiple = False
print('Begining to load')
gen_final.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, num_loaded, limited_data=True, multiple=multiple)
print(gen_final.X_train.shape)
print(gen_final.Y_train.shape)

print('Initializing model')
gen_final.set_model_to_final_model(multiple=multiple)

# The training
name = 'SUPER_limited_Final_Model_singleOutput'
gen_final.train_model(name = name)


In [ ]:
# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seqLleng = 100 
n_test = 0
num_loaded = len(joint_angle_data_path_list)+1
multiple = True
print('Begining to load')
gen_final.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, num_loaded, limited_data=True, multiple=multiple)
print(gen_final.X_train.shape)
print(gen_final.Y_train.shape)

print('Initializing model')
gen_final.set_model_to_final_model(multiple=multiple)

# The training
name = 'SUPER_limited_Final_Model_multipleOutput'
gen_final.train_model(name = name)


In [ ]:
# Load in all training data and save as an pickle
joint_angle_data_path_list = []
power_data_path_list = []

for name in os.listdir(resource_path):
    path = os.path.join(resource_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

print('Number of paths to load: ' + str(len(joint_angle_data_path_list)))
gen_final = Generator()
seqLleng = 100 
n_test = 0
num_loaded = len(joint_angle_data_path_list)+1
multiple = False
print('Begining to load')
gen_final.set_up_multi_data(seqLleng, joint_angle_data_path_list, power_data_path_list, match_list_vector, num_loaded, limited_data=True, multiple=multiple)
print(gen_final.X_train.shape)
print(gen_final.Y_train.shape)

print('Initializing model')
gen_final.set_model_to_FFNN()

# The training
name = 'SUPER_limited_Final_Model_FNN'
gen_final.train_model(name = name)


## Test set prediction using the final model

In [ ]:
seqLleng = 100 #12ms/leng unit
multiple = False

test_set_path = 'Pickled_data/test_sets'

joint_angle_data_path_list = []
power_data_path_list = []

# Load all the test sets
for name in os.listdir(test_set_path):
    path = os.path.join(test_set_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

plt.figure(figsize=(12,12))
fig = plt.subplot(nrows=3, ncols=3)        
        
for n in range(9):
    fig.subplot(n)
    print('Working with path number: ' + str(n+1) + ' of 9')
    gen_final = Generator()
    gen_final.load_pre_processed_data(joint_angle_data_path_list[n],power_data_path_list[match_list_vector[n]])
    num_loaded = 2
    gen_final.set_up_multi_data(seqLleng, [joint_angle_data_path_list[n]], [power_data_path_list[match_list_vector[n]]], [0], num_loaded, multiple=multiple)
    gen_final.set_model_to_final_model(multiple=multiple)
    gen_final.pre_train()
    gen_final.model.load_weights('data/weights/model_Final_Model_singleOut_Iteration_2_trainingNum_42.hdf5')
    print('Prediction ')
    predictions_test = gen_final.model.predict(gen_final.X_train)
    # Figure plotting
    x = range(len(predictions_test))
    fig.plot(x[100:1000], gen_final.Y_train[100:1000],  label='true',  marker='o',linewidth=0.3, markersize=4)
    fig.plot(x[100:1000], predictions_test[100:1000],  label='pred',  marker='o',linewidth=0.3, markersize=4)
    fig.legend();
    fig.set_ylabel('Power [W]')
    fig.set_xlabel('Time [s]')
    fig.set_title('Test data prediction');

## Plot Figures  (Just some test examples)

In [ ]:
#predictions_test = gen_final.model.predict(gen_final.X_train)

plt.figure(figsize=(12,6))
fig = plt.subplot(3,3)

subplot(1)
x = range(len(predictions_test))
fig.plot(x[100:1000], gen_final.Y_train[100:1000],  label='true',  marker='o',linewidth=0.3, markersize=4)
fig.plot(x[100:1000], predictions_test[100:1000],  label='pred',  marker='o',linewidth=0.3, markersize=4)
fig.legend();
fig.set_ylabel('Power [W]')
fig.set_xlabel('Time [s]')
fig.set_title('Test data prediction');

In [ ]:
prediction_paths = []
# Load all the test sets
for name in os.listdir(test_set_path):
    path = os.path.join(test_set_path, name)
    if "_auginput.pickle" in name:
        joint_angle_data_path_list.append(path)
    if "_poweroutput.pickle" in name:
        power_data_path_list.append(path)

match_list_vector = []
for filename in joint_angle_data_path_list:
    name = filename[:-len("_auginput.pickle")]
    num = 0
    for filename in power_data_path_list:
        if name in filename:
            match_list_vector.append(num)
            continue
        num = num + 1

In [ ]:
print(test_set_path)